In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 

In [2]:
train = pd.read_csv('Data/train.csv')
test  = pd.read_csv('Data/test.csv')

x_train = train.drop(columns=['target']).values
y_train = train['target'].values

x_test = test.drop(columns=['target']).values
y_test = test['target'].values

In [3]:
train.columns

Index(['tcp.flags', 'tcp.time_delta', 'tcp.len', 'mqtt.conack.flags',
       'mqtt.conack.val', 'mqtt.conflag.passwd', 'mqtt.conflags',
       'mqtt.dupflag', 'mqtt.hdrflags', 'mqtt.kalive', 'mqtt.len',
       'mqtt.msgid', 'mqtt.msgtype', 'mqtt.retain', 'target'],
      dtype='object')

# 0. Feature Scaling

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [5]:
import Resampler as res
import Metrics as met

dist = res.class_distribution(y_train)
dist

[(0, 10170), (1, 428), (2, 8340692), (3, 91465), (4, 7637), (5, 6433)]

In [7]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(128, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(64, kernel_initializer='normal'))
    model.add(keras.layers.Dense(6,activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

monitor = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

# 1. No Resampling

In [8]:
model = build_model()
model.fit(x_train,y_train,
        validation_data=(x_test,y_test),
        callbacks=[monitor],
        verbose=2,
        epochs=20,
        batch_size=4096)

2022-12-02 19:52:22.891187: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-02 19:52:22.891361: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/20


2022-12-02 19:52:23.362924: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-02 19:52:23.964247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 19:52:43.363994: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2065/2065 - 25s - loss: 0.0463 - accuracy: 0.9940 - val_loss: 0.0292 - val_accuracy: 0.9951 - 25s/epoch - 12ms/step
Epoch 2/20
2065/2065 - 25s - loss: 0.0284 - accuracy: 0.9952 - val_loss: 0.0282 - val_accuracy: 0.9952 - 25s/epoch - 12ms/step
Epoch 3/20
2065/2065 - 24s - loss: 0.0280 - accuracy: 0.9953 - val_loss: 0.0280 - val_accuracy: 0.9952 - 24s/epoch - 11ms/step
Epoch 4/20
2065/2065 - 26s - loss: 0.0279 - accuracy: 0.9953 - val_loss: 0.0281 - val_accuracy: 0.9952 - 26s/epoch - 12ms/step
Epoch 5/20
2065/2065 - 25s - loss: 0.0278 - accuracy: 0.9953 - val_loss: 0.0280 - val_accuracy: 0.9953 - 25s/epoch - 12ms/step
Epoch 6/20
2065/2065 - 23s - loss: 0.0277 - accuracy: 0.9953 - val_loss: 0.0279 - val_accuracy: 0.9952 - 23s/epoch - 11ms/step
Epoch 7/20
2065/2065 - 23s - loss: 0.0277 - accuracy: 0.9953 - val_loss: 0.0279 - val_accuracy: 0.9954 - 23s/epoch - 11ms/step
Epoch 7: early stopping


In [9]:
met.eval_tf(model, x_train, x_test, y_train, y_test)

  35/1033 [>.............................] - ETA: 4s

2022-12-02 19:55:15.470748: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1033/1033 [==============================] - 5s 5ms/step
Training Result:  Accuracy: 0.9954 F1 Score: [0.6612, 0.6562, 0.9979, 0.8353, 0.4947, 0.5811]
443/443 [==============================] - 2s 5ms/step
Testing  Result:  Accuracy: 0.9954 F1 Score: [0.6463, 0.6347, 0.9979, 0.8326, 0.4798, 0.5594]


## 2. Undersampling + Oversampling

In [10]:
n_majority = int(x_train.shape[0] * 0.7)

n_minorities = np.array(dist)[[0,1,3,4,5],1]
weights = n_minorities / n_minorities.sum()
n_minorities = (weights * n_majority).astype(int)

undersample_strategy = {2: n_majority}
oversample_strategy = {
    0: n_minorities[0],
    1: n_minorities[1],
    3: n_minorities[2],
    4: n_minorities[3],
    5: n_minorities[4],
}


In [14]:
# x_resampled, y_resampled = res.random_undersample(x_train, y_train, strategy=undersample_strategy)

# x_resampled, y_resampled = res.random_oversample(x_resampled, y_resampled, strategy=oversample_strategy)

import pickle 

with open('underover.pkl', 'rb') as file:
    x_resampled, y_resampled = pickle.load(file)

res.class_distribution(y_resampled)

In [17]:
model2 = build_model()
model2.fit(x_resampled, y_resampled,
        validation_data=(x_test,y_test),
        callbacks=[monitor],
        verbose=2,
        epochs=20,
        batch_size=4096)

Epoch 1/20


2022-12-02 21:17:20.760643: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 21:17:49.054804: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2891/2891 - 35s - loss: 0.4266 - accuracy: 0.8261 - val_loss: 0.2269 - val_accuracy: 0.9953 - 35s/epoch - 12ms/step
Epoch 2/20
2891/2891 - 33s - loss: 0.4030 - accuracy: 0.8312 - val_loss: 0.2259 - val_accuracy: 0.9953 - 33s/epoch - 11ms/step
Epoch 3/20
2891/2891 - 36s - loss: 0.4022 - accuracy: 0.8314 - val_loss: 0.2266 - val_accuracy: 0.9953 - 36s/epoch - 12ms/step
Epoch 4/20
2891/2891 - 31s - loss: 0.4018 - accuracy: 0.8314 - val_loss: 0.2157 - val_accuracy: 0.9953 - 31s/epoch - 11ms/step
Epoch 5/20
2891/2891 - 31s - loss: 0.4016 - accuracy: 0.8315 - val_loss: 0.2218 - val_accuracy: 0.9953 - 31s/epoch - 11ms/step
Epoch 6/20
2891/2891 - 31s - loss: 0.4014 - accuracy: 0.8315 - val_loss: 0.2177 - val_accuracy: 0.9953 - 31s/epoch - 11ms/step
Epoch 7/20
2891/2891 - 31s - loss: 0.4013 - accuracy: 0.8315 - val_loss: 0.2251 - val_accuracy: 0.9953 - 31s/epoch - 11ms/step
Epoch 8/20
2891/2891 - 31s - loss: 0.4011 - accuracy: 0.8316 - val_loss: 0.2165 - val_accuracy: 0.9953 - 31s/epoch - 11ms/

In [18]:
met.eval_tf(model2, x_resampled, x_test, y_resampled, y_test)

  31/1446 [..............................] - ETA: 7s

2022-12-02 21:22:11.150337: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1446/1446 [==============================] - 7s 5ms/step
Training Result:  Accuracy: 0.8317 F1 Score: [0.6387, 0.662, 0.8688, 0.8357, 0.5124, 0.472]
443/443 [==============================] - 2s 5ms/step
Testing  Result:  Accuracy: 0.9953 F1 Score: [0.6239, 0.6397, 0.9979, 0.8326, 0.4887, 0.4545]


## 3. Undersampling + SMOTE (Synthetic Minority Over-sampling )

In [19]:
# x_resampled, y_resampled = res.random_undersample(x_train, y_train, strategy=undersample_strategy)

# x_resampled, y_resampled = res.smote(x_resampled, y_resampled, strategy=oversample_strategy)

with open('undersmote.pkl', 'rb') as file:
    x_resampled, y_resampled = pickle.load(file)

res.class_distribution(y_resampled)

[(0, 518406), (1, 21816), (2, 5919777), (3, 4662347), (4, 389289), (5, 327916)]

In [20]:
model3 = build_model()
model3.fit(x_resampled, y_resampled,
        validation_data=(x_test,y_test),
        callbacks=[monitor],
        verbose=2,
        epochs=20,
        batch_size=4096)

Epoch 1/20


2022-12-02 21:23:10.953482: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 21:23:40.506225: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2891/2891 - 36s - loss: 0.4255 - accuracy: 0.8261 - val_loss: 0.2288 - val_accuracy: 0.9952 - 36s/epoch - 12ms/step
Epoch 2/20
2891/2891 - 31s - loss: 0.4032 - accuracy: 0.8311 - val_loss: 0.2239 - val_accuracy: 0.9952 - 31s/epoch - 11ms/step
Epoch 3/20
2891/2891 - 31s - loss: 0.4024 - accuracy: 0.8312 - val_loss: 0.2341 - val_accuracy: 0.9953 - 31s/epoch - 11ms/step
Epoch 4/20
2891/2891 - 31s - loss: 0.4021 - accuracy: 0.8312 - val_loss: 0.2162 - val_accuracy: 0.9953 - 31s/epoch - 11ms/step
Epoch 5/20
2891/2891 - 31s - loss: 0.4019 - accuracy: 0.8312 - val_loss: 0.2202 - val_accuracy: 0.9953 - 31s/epoch - 11ms/step
Epoch 6/20
2891/2891 - 31s - loss: 0.4017 - accuracy: 0.8313 - val_loss: 0.2346 - val_accuracy: 0.9953 - 31s/epoch - 11ms/step
Epoch 7/20
2891/2891 - 31s - loss: 0.4015 - accuracy: 0.8313 - val_loss: 0.2299 - val_accuracy: 0.9953 - 31s/epoch - 11ms/step
Epoch 8/20
2891/2891 - 32s - loss: 0.4015 - accuracy: 0.8313 - val_loss: 0.2271 - val_accuracy: 0.9953 - 32s/epoch - 11ms/

In [21]:
met.eval_tf(model3, x_resampled, x_test, y_resampled, y_test)

  34/1446 [..............................] - ETA: 6s

2022-12-02 21:27:55.177817: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1446/1446 [==============================] - 7s 5ms/step
Training Result:  Accuracy: 0.8314 F1 Score: [0.6374, 0.6625, 0.8686, 0.8356, 0.5088, 0.4706]
443/443 [==============================] - 2s 5ms/step
Testing  Result:  Accuracy: 0.9953 F1 Score: [0.6245, 0.63, 0.9979, 0.8326, 0.4882, 0.4545]


## 4. NearMiss + SMOTE

In [22]:
# x_resampled, y_resampled = res.near_miss(x_train, y_train, strategy=undersample_strategy)

# x_resampled, y_resampled = res.smote(x_resampled, y_resampled, strategy=oversample_strategy)

with open('nmsmote.pkl', 'rb') as file:
    x_resampled, y_resampled = pickle.load(file)

res.class_distribution(y_resampled)

[(0, 518406), (1, 21816), (2, 5919777), (3, 4662347), (4, 389289), (5, 327916)]

In [23]:
model4 = build_model()
model4.fit(x_resampled, y_resampled,
        validation_data=(x_test,y_test),
        callbacks=[monitor],
        verbose=2,
        epochs=20,
        batch_size=4096)

Epoch 1/20


2022-12-02 21:30:02.145017: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 21:30:28.637795: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2891/2891 - 32s - loss: 0.4653 - accuracy: 0.8264 - val_loss: 0.4966 - val_accuracy: 0.9898 - 32s/epoch - 11ms/step
Epoch 2/20
2891/2891 - 31s - loss: 0.4427 - accuracy: 0.8310 - val_loss: 0.6065 - val_accuracy: 0.9898 - 31s/epoch - 11ms/step
Epoch 3/20
2891/2891 - 30s - loss: 0.4420 - accuracy: 0.8311 - val_loss: 0.6720 - val_accuracy: 0.9886 - 30s/epoch - 11ms/step
Epoch 4/20
2891/2891 - 31s - loss: 0.4416 - accuracy: 0.8312 - val_loss: 0.7281 - val_accuracy: 0.9877 - 31s/epoch - 11ms/step
Epoch 5/20
2891/2891 - 30s - loss: 0.4414 - accuracy: 0.8313 - val_loss: 0.8092 - val_accuracy: 0.9878 - 30s/epoch - 11ms/step
Epoch 6/20
2891/2891 - 31s - loss: 0.4413 - accuracy: 0.8313 - val_loss: 0.9794 - val_accuracy: 0.9386 - 31s/epoch - 11ms/step
Epoch 6: early stopping


In [25]:
met.eval_tf(model4, x_resampled, x_test, y_resampled, y_test)

1446/1446 [==============================] - 7s 5ms/step
Training Result:  Accuracy: 0.8315 F1 Score: [0.6368, 0.6611, 0.8686, 0.8356, 0.5118, 0.4706]
443/443 [==============================] - 2s 5ms/step
Testing  Result:  Accuracy: 0.9386 F1 Score: [0.624, 0.6397, 0.9683, 0.8327, 0.0103, 0.4545]
